In [71]:
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.tree import DecisionTreeClassifier

In [72]:
caracteristicas=8
instancias=10
X, Y = make_multilabel_classification(n_samples=instancias, n_features=caracteristicas, n_classes=5)

In [73]:
X

array([[  3.,   3.,   9.,  10.,   4.,   6.,   1.,  10.],
       [  2.,   9.,   2.,   7.,   8.,   4.,   6.,   2.],
       [  8.,   7.,  12.,   8.,   2.,  13.,   4.,   5.],
       [  6.,   5.,   5.,   3.,   8.,  12.,   6.,   1.],
       [  0.,   7.,   5.,  16.,  14.,  10.,   7.,   3.],
       [  4.,   2.,   9.,   0.,   7.,  14.,   7.,   6.],
       [  3.,   8.,   9.,  13.,   2.,  13.,   4.,   2.],
       [  2.,  10.,   9.,   9.,   6.,   5.,   4.,   6.],
       [  1.,   2.,   8.,   7.,  20.,   2.,   5.,   8.],
       [  4.,   4.,   5.,   8.,  15.,  10.,   3.,   2.]])

In [74]:
Y

array([[0, 1, 0, 1, 0],
       [1, 1, 1, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 1, 0, 1, 1],
       [1, 0, 1, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 1]])

In [75]:
rnd_dimensions=np.random.randint(0, 2,caracteristicas)
rnd_dimensions=rnd_dimensions.astype(bool)
rnd_dimensions

array([ True, False, False,  True,  True,  True,  True,  True], dtype=bool)

In [76]:
s=list(range(instancias))
shuffle(s)
rnd_neighbors=np.array(s[:(int(len(s)/2))])
rnd_neighbors

array([7, 0, 6, 5, 9])

In [77]:
m_reducida=X[:, rnd_dimensions]
m_reducida

array([[  3.,  10.,   4.,   6.,   1.,  10.],
       [  2.,   7.,   8.,   4.,   6.,   2.],
       [  8.,   8.,   2.,  13.,   4.,   5.],
       [  6.,   3.,   8.,  12.,   6.,   1.],
       [  0.,  16.,  14.,  10.,   7.,   3.],
       [  4.,   0.,   7.,  14.,   7.,   6.],
       [  3.,  13.,   2.,  13.,   4.,   2.],
       [  2.,   9.,   6.,   5.,   4.,   6.],
       [  1.,   7.,  20.,   2.,   5.,   8.],
       [  4.,   8.,  15.,  10.,   3.,   2.]])

In [78]:
m_vecinos=np.zeros((instancias,len(rnd_neighbors)))
m_vecinos

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [79]:
cont=-1
for i in m_reducida:
    dist=999
    cont+=1
    cont2=-1
    for j in rnd_neighbors:
        cont2+=1
        dist2=euclidean_distances([i],[m_reducida[j,:]])
        if dist2<dist:
            dist=dist2
            a=cont
            b=cont2
    m_vecinos[a][b]=1
m_vecinos

array([[ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [80]:
m_entrenamiento=np.concatenate((X,m_vecinos),axis=1)
m_entrenamiento

array([[  3.,   3.,   9.,  10.,   4.,   6.,   1.,  10.,   0.,   1.,   0.,
          0.,   0.],
       [  2.,   9.,   2.,   7.,   8.,   4.,   6.,   2.,   1.,   0.,   0.,
          0.,   0.],
       [  8.,   7.,  12.,   8.,   2.,  13.,   4.,   5.,   0.,   0.,   1.,
          0.,   0.],
       [  6.,   5.,   5.,   3.,   8.,  12.,   6.,   1.,   0.,   0.,   0.,
          1.,   0.],
       [  0.,   7.,   5.,  16.,  14.,  10.,   7.,   3.,   0.,   0.,   0.,
          0.,   1.],
       [  4.,   2.,   9.,   0.,   7.,  14.,   7.,   6.,   0.,   0.,   0.,
          1.,   0.],
       [  3.,   8.,   9.,  13.,   2.,  13.,   4.,   2.,   0.,   0.,   1.,
          0.,   0.],
       [  2.,  10.,   9.,   9.,   6.,   5.,   4.,   6.,   1.,   0.,   0.,
          0.,   0.],
       [  1.,   2.,   8.,   7.,  20.,   2.,   5.,   8.,   0.,   0.,   0.,
          0.,   1.],
       [  4.,   4.,   5.,   8.,  15.,  10.,   3.,   2.,   0.,   0.,   0.,
          0.,   1.]])

In [81]:
e=DecisionTreeClassifier()
e.fit(m_entrenamiento,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [132]:
m_predecir=np.zeros((len(rnd_neighbors),caracteristicas))
cont=0
for i in range(len(X)):
    if i in rnd_neighbors:
        m_predice[cont]=X[i,:]
        cont+=1
m_predicir

array([[  3.,   3.,   9.,  10.,   4.,   6.,   1.,  10.],
       [  4.,   2.,   9.,   0.,   7.,  14.,   7.,   6.],
       [  3.,   8.,   9.,  13.,   2.,  13.,   4.,   2.],
       [  2.,  10.,   9.,   9.,   6.,   5.,   4.,   6.],
       [  4.,   4.,   5.,   8.,  15.,  10.,   3.,   2.]])

In [539]:
e.predict(X)
print(e)

ValueError: Number of features of the model must match the input. Model n_features is 13 and input n_features is 8 